<a href="https://colab.research.google.com/github/AnguillaJaponica/decision_analyzer/blob/master/chapter3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import libraries
import numpy as np
import pandas as pd
# DataFrameの全角文字の出力をきれいにする。
pd.set_option('display.unicode.east_asian_width', True)

In [2]:
# 利得を計算する際のパラメータ
fixed_cost = 100
run_cost = 600
sales_price = 0.2

machine_ability = 5000
demand_boom = 10000
demand_slump = 5000

In [3]:
# 出荷される製品の個数
num_product_df = pd.DataFrame({
    '0台': [0, 0],
    '1台': [min([machine_ability, demand_boom]),
           min([machine_ability, demand_slump])],
    '2台': [min([machine_ability * 2, demand_boom]),
           min([machine_ability * 2, demand_slump])]
})
num_product_df.index = ['好況', '不況']
print(num_product_df)

      0台   1台    2台
好況    0  5000  10000
不況    0  5000   5000


In [4]:
# 売上行列
sales_df = num_product_df * sales_price
print(sales_df)

      0台     1台     2台
好況  0.0  1000.0  2000.0
不況  0.0  1000.0  1000.0


In [5]:
# 製造コスト
run_cost_df = pd.DataFrame({
    '0台': np.repeat(fixed_cost, 2),
    '1台': np.repeat(fixed_cost + run_cost, 2),
    '2台': np.repeat(fixed_cost + run_cost * 2, 2)
})
run_cost_df.index =  ['好況', '不況']
print(run_cost_df)

      0台  1台   2台
好況  100  700  1300
不況  100  700  1300


In [6]:
# 利得行列
paypff_df = sales_df - run_cost_df
print(paypff_df)

        0台    1台    2台
好況 -100.0  300.0  700.0
不況 -100.0  300.0 -300.0


In [7]:
# 利得行列を一発で作る関数

def calc_payoff_table(fixed_cost, run_cost, sales_price, machine_ability, demand_boom ,demand_slump):
  # 出荷される製品の個数
  num_product_df = pd.DataFrame({
      '0台': [0, 0],
      '1台': [min([machine_ability, demand_boom]),
            min([machine_ability, demand_slump])],
      '2台': [min([machine_ability * 2, demand_boom]),
            min([machine_ability * 2, demand_slump])]
  })
  sales_df = num_product_df * sales_price
  # 製造コスト
  run_cost_df = pd.DataFrame({
      '0台': np.repeat(fixed_cost, 2),
      '1台': np.repeat(fixed_cost + run_cost, 2),
      '2台': np.repeat(fixed_cost + run_cost * 2, 2)
  })
  # 利得行列
  paypff_df = sales_df - run_cost_df
  paypff_df.index =  ['好況', '不況']
  # 結果を返す
  return(paypff_df)


In [8]:
payoff = calc_payoff_table(fixed_cost = 100, run_cost = 600, sales_price = 0.2, machine_ability = 5000, demand_boom = 10000 ,demand_slump = 5000)

In [9]:
# 選択肢ごとの最大利得
payoff.max()

0台   -100.0
1台    300.0
2台    700.0
dtype: float64

In [22]:
payoff.max().max()

700.0

In [10]:
payoff.max() == payoff.max().max()

0台    False
1台    False
2台     True
dtype: bool

In [11]:
payoff.max()[payoff.max() == payoff.max().max()]

2台    700.0
dtype: float64

In [12]:
list(payoff.max()[payoff.max() == payoff.max().max()].index)

['2台']

In [17]:
# 最大値を取るインデックスを取得する。最大値が複数ある場合は全て取得する。
def argmax_list(series):
  return(list(series[series == series.max()].index))

# 最小値を取るインデックスを取得する。最小値が複数ある場合は全て取得する。
def argmin_list(series):
  return(list(series[series == series.min()].index))

In [16]:
print('Maximax: ', argmax_list(payoff.max()))

Maximax:  ['2台']


In [18]:
# 選択肢ごとの最小利得
payoff.min()

0台   -100.0
1台    300.0
2台   -300.0
dtype: float64

In [19]:
print('Maximin: ', argmax_list(payoff.min()))

Maximin:  ['1台']


In [20]:
# ハーヴィッツの基準
# alpha=0.6の場合
hurwicz = payoff.max() * 0.6 + payoff.min() * 0.4
hurwicz

0台   -100.0
1台    300.0
2台    300.0
dtype: float64

In [21]:
argmax_list(hurwicz)

['1台', '2台']

In [22]:
# ハーヴィッツの基準による決定を行う関数
def hurwicz(payoff_table, alpha):
  hurwicz = payoff_table.max() * alpha + payoff_table.min() * (1 - alpha)
  return argmax_list(hurwicz)

In [26]:
print('Hurwicz: ', hurwicz(payoff, 0.6))

Hurwicz:  ['1台', '2台']


In [27]:
print('Hurwicz: ', hurwicz(payoff, 0.7))

Hurwicz:  ['2台']


In [28]:
print('Hurwicz: ', hurwicz(payoff, 0.5))

Hurwicz:  ['1台']


In [57]:
payoff

,0台,1台,2台
好況,-100.0,300.0,700.0
不況,-100.0,300.0,-300.0


In [60]:
# 自然の状態ごとに見た最大利得
best_df = pd.concat([payoff.max(axis=1)] * payoff.shape[1], axis = 1)
best_df.columns = payoff.columns
print(best_df)

        0台    1台    2台
好況  700.0  700.0  700.0
不況  300.0  300.0  300.0


In [61]:
# リグレット
regret_df = best_df - payoff
print(regret_df)

        0台    1台    2台
好況  800.0  400.0    0.0
不況  400.0    0.0  600.0


In [62]:
# 各々の選択肢におけるリグレットの最大値
regret_df.max()

0台    800.0
1台    400.0
2台    600.0
dtype: float64

In [63]:
# ミニマックスリグレットは、リグレットの最大値が最小になるような選択肢を採択する
argmin_list(regret_df.max())

['1台']

In [64]:
# ミニマックスリグレット基準による決定を行う関数
def minimax_regret(payoff_table):
  best_df = pd.concat([payoff_table.max(axis=1)] * payoff_table.shape[1], axis = 1)
  best_df.columns = payoff_table.columns
  regret_df = best_df - payoff
  return argmin_list(regret_df.max())

In [65]:
minimax_regret(payoff)

['1台']

In [66]:
# 選択肢ごとの平均利得
payoff.mean()

0台   -100.0
1台    300.0
2台    200.0
dtype: float64

In [67]:
print('Laplace: ', argmax_list(payoff.mean()))

Laplace:  ['1台']


In [68]:
# 感度分析
# 機械1台の稼働コストを増やした
payoff_2 = calc_payoff_table(fixed_cost = 100, run_cost = 625, sales_price = 0.2, machine_ability = 5000, demand_boom = 10000 ,demand_slump = 5000)

print(payoff_2)

        0台    1台    2台
好況 -100.0  275.0  650.0
不況 -100.0  275.0 -350.0


In [69]:
# 機械1台の稼働コストを減らした
payoff_3 = calc_payoff_table(fixed_cost = 100, run_cost = 575, sales_price = 0.2, machine_ability = 5000, demand_boom = 10000 ,demand_slump = 5000)

print(payoff_3)

        0台    1台    2台
好況 -100.0  325.0  750.0
不況 -100.0  325.0 -250.0


In [70]:
minimax_regret(payoff_2)

['1台']

In [71]:
minimax_regret(payoff_3)

['1台']

In [79]:
# 機械1台の稼働コストをさらに減らした
payoff_4 = calc_payoff_table(fixed_cost = 100, run_cost = 300, sales_price = 0.2, machine_ability = 5000, demand_boom = 10000 ,demand_slump = 5000)

print(payoff_4)

        0台    1台     2台
好況 -100.0  600.0  1300.0
不況 -100.0  600.0   300.0


In [80]:
minimax_regret(payoff_4)

['2台']